In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Flatten, Dense, Dropout
import warnings
sns.set()
%matplotlib inline
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
df_train = pd.read_csv('/kaggle/input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
df_test = pd.read_csv('/kaggle/input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv')

In [6]:
y_train = df_train['label']
y_test = df_test['label']
df_train.drop(['label'], axis=1, inplace=True)
df_test.drop(['label'], axis=1, inplace=True)

In [7]:
size  = 28
channels = 1
batch = 128
epochs = 10

In [8]:
X_train = df_train.values.reshape(df_train.shape[0], size, size, channels)
X_test = df_test.values.reshape(df_test.shape[0], size, size, channels)

In [9]:
df_train

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,107,118,127,134,139,143,146,150,153,156,...,207,207,207,207,206,206,206,204,203,202
1,155,157,156,156,156,157,156,158,158,157,...,69,149,128,87,94,163,175,103,135,149
2,187,188,188,187,187,186,187,188,187,186,...,202,201,200,199,198,199,198,195,194,195
3,211,211,212,212,211,210,211,210,210,211,...,235,234,233,231,230,226,225,222,229,163
4,164,167,170,172,176,179,180,184,185,186,...,92,105,105,108,133,163,157,163,164,179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27450,189,189,190,190,192,193,193,193,193,194,...,132,165,99,77,52,200,234,200,222,225
27451,151,154,157,158,160,161,163,164,166,167,...,198,198,198,198,198,196,195,195,195,194
27452,174,174,174,174,174,175,175,174,173,173,...,121,196,209,208,206,204,203,202,200,200
27453,177,181,184,185,187,189,190,191,191,190,...,119,56,27,58,102,79,47,64,87,93


In [10]:
checkpoint_filepath = 'best_model.hdf5'

callback_checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=False, monitor='val_accuracy', mode='max', save_best_only=True)
callback_learningrate = ReduceLROnPlateau(monitor='loss', mode='min', min_delta=0.01, patience=3, factor=.75, min_lr=0.00001, verbose=1)

callbacks = [callback_checkpoint, callback_learningrate]

In [11]:
Model = Sequential([Conv2D(filters=32,  kernel_size=(3,3), activation="relu", input_shape=(size,size,channels)),
                    MaxPool2D(2,2, padding='same'),
                    Dropout(0.2),
                 
                    Conv2D(filters=128,  kernel_size=(3,3), activation="relu"),
                    MaxPool2D(2,2, padding='same'),
                    Dropout(0.2),
                
                    Conv2D(filters=512, kernel_size=(3,3), activation="relu"),
                    MaxPool2D(2,2, padding='same'),
                    Dropout(0.2),
                    
                    
                    Flatten(),
                    
                    Dense(units=4096, activation="relu"),                 
                    Dropout(0.2),
                    
                    Dense(units=1024, activation="relu"),
                    Dropout(0.2),
                                  
                    Dense(units=256, activation="relu"),
                    Dropout(0.2),
                    
                    Dense(units=25, activation="softmax"),
])


Model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=["accuracy"])


Model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 13, 13, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 128)       36992     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 128)        0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 6, 6, 128)         0

In [12]:
type(X_test)

numpy.ndarray

In [13]:
X_train.shape

(27455, 28, 28, 1)

In [14]:
y = y_train.values

In [15]:
type(y_train)

pandas.core.series.Series

In [16]:
history = Model.fit(X_train,y, epochs=epochs, callbacks=callbacks)

Epoch 1/10


2023-05-07 16:42:32.845595: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


858/858 [==============================] - 13s 6ms/step - loss: 2.0954 - accuracy: 0.4243 - lr: 0.0010
Epoch 2/10
858/858 [==============================] - 5s 6ms/step - loss: 0.3433 - accuracy: 0.8889 - lr: 0.0010
Epoch 3/10
858/858 [==============================] - 5s 6ms/step - loss: 0.1628 - accuracy: 0.9564 - lr: 0.0010
Epoch 4/10
858/858 [==============================] - 5s 6ms/step - loss: 0.1234 - accuracy: 0.9670 - lr: 0.0010
Epoch 5/10
858/858 [==============================] - 5s 6ms/step - loss: 0.1267 - accuracy: 0.9718 - lr: 0.0010
Epoch 6/10
858/858 [==============================] - 5s 6ms/step - loss: 0.1325 - accuracy: 0.9705 - lr: 0.0010
Epoch 7/10
858/858 [==============================] - 5s 6ms/step - loss: 0.1038 - accuracy: 0.9784 - lr: 0.0010
Epoch 8/10
858/858 [==============================] - 5s 6ms/step - loss: 0.1160 - accuracy: 0.9791 - lr: 0.0010
Epoch 9/10
858/858 [==============================] - 5s 6ms/step - loss: 0.1363 - accuracy: 0.9769 - lr: 

In [17]:
predection = Model.predict(X_test)

225/225 [==============================] - 1s 2ms/step


In [18]:
predection

array([[0.0000000e+00, 0.0000000e+00, 5.9192651e-25, ..., 0.0000000e+00,
        1.2010470e-29, 0.0000000e+00],
       [3.0301167e-29, 1.8212006e-28, 1.9784900e-32, ..., 1.1520999e-30,
        3.2573957e-36, 1.8755217e-23],
       [1.6514695e-33, 8.7047180e-28, 0.0000000e+00, ..., 4.3126099e-24,
        5.6801081e-35, 3.5072517e-31],
       ...,
       [5.2868709e-29, 5.4433749e-33, 1.0000000e+00, ..., 4.2240115e-21,
        4.2252640e-23, 1.6619030e-26],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, ..., 3.7735412e-31,
        1.3029092e-36, 0.0000000e+00]], dtype=float32)

In [19]:
predection.shape

(7172, 25)

In [20]:
pred = [np.argmax(i) for i in predection]

In [21]:
pred_letter = []
x = {0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',10:'K',11:'L',12:'M',13:'N',14:'O',15:'P',16:'Q',17:'R',18:'S',19:'T',20:'U',21:'V',22:'W',23:'X',24:'Y'}
for i in pred:
    letter = x[i]
    pred_letter.append(letter)
    
    

In [23]:
pred_letter

['G',
 'F',
 'K',
 'A',
 'D',
 'V',
 'K',
 'O',
 'D',
 'H',
 'I',
 'I',
 'V',
 'M',
 'H',
 'E',
 'W',
 'A',
 'H',
 'H',
 'C',
 'A',
 'V',
 'E',
 'K',
 'P',
 'O',
 'P',
 'H',
 'B',
 'H',
 'I',
 'N',
 'T',
 'D',
 'V',
 'A',
 'D',
 'S',
 'O',
 'P',
 'X',
 'I',
 'P',
 'O',
 'F',
 'R',
 'E',
 'X',
 'N',
 'U',
 'W',
 'D',
 'F',
 'Q',
 'Q',
 'V',
 'E',
 'G',
 'W',
 'K',
 'N',
 'L',
 'W',
 'C',
 'K',
 'B',
 'E',
 'S',
 'E',
 'U',
 'G',
 'P',
 'E',
 'D',
 'U',
 'P',
 'L',
 'C',
 'C',
 'R',
 'C',
 'H',
 'V',
 'X',
 'H',
 'M',
 'H',
 'I',
 'O',
 'C',
 'B',
 'H',
 'T',
 'I',
 'F',
 'A',
 'A',
 'T',
 'V',
 'I',
 'E',
 'C',
 'U',
 'Q',
 'B',
 'P',
 'O',
 'O',
 'G',
 'M',
 'F',
 'A',
 'Y',
 'C',
 'X',
 'O',
 'Y',
 'Q',
 'K',
 'E',
 'I',
 'I',
 'E',
 'M',
 'I',
 'G',
 'V',
 'E',
 'P',
 'L',
 'V',
 'C',
 'P',
 'X',
 'X',
 'E',
 'V',
 'W',
 'D',
 'P',
 'S',
 'H',
 'R',
 'V',
 'G',
 'B',
 'G',
 'K',
 'F',
 'E',
 'V',
 'D',
 'V',
 'O',
 'X',
 'C',
 'K',
 'H',
 'F',
 'O',
 'V',
 'I',
 'E',
 'S',
 'X',
 'N'